In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import requests
import plotly.express as px
from urllib.request import urlopen
import json
import plotly.figure_factory as ff

In [2]:
df = pd.read_csv('../StartingWithToday/data/joined_depression_cre_walkability.csv')
    
df.columns = df.columns.str.lower()
    
df.rename(columns={'indicator rate value':'depression_rate','locationname':'geoid_tract_20'},inplace=True)

Describe the data to identify potentially incorrect values.

In [3]:
df[['blw_pov_lvl_pe','depression_rate','no_health_ins_pe','gini_ind_inequality_e',
    'no_veh_pe','hs_grad_pe','broadband_pe','median_natwalkind']].describe().T

,count,mean,std,min,25%,50%,75%,max
blw_pov_lvl_pe,396.0,1.242525e+01,1.072242e+01,2.000000e-01,5.100000,9.100000,15.825000,68.600000
depression_rate,396.0,1.628737e+01,3.868286e+00,1.070000e+01,13.000000,14.500000,19.725000,30.000000
no_health_ins_pe,396.0,7.207071e+00,7.166621e+00,0.000000e+00,2.775000,5.000000,9.200000,56.100000
gini_ind_inequality_e,396.0,-1.683501e+06,3.350126e+07,-6.666667e+08,0.356125,0.404700,0.471150,0.673000
no_veh_pe,396.0,-1.683481e+06,3.350126e+07,-6.666667e+08,5.100000,15.700000,33.250000,83.000000
hs_grad_pe,396.0,8.822197e+01,1.011174e+01,1.830000e+01,84.175000,90.600000,95.000000,100.000000
broadband_pe,396.0,-1.683418e+06,3.350126e+07,-6.666667e+08,78.100000,86.900000,92.200000,100.000000
median_natwalkind,396.0,1.261553e+01,3.245840e+00,3.166667e+00,11.062500,13.208333,14.833333,18.833333


Replace any values less than 0 as `NaN`. These features are not expected to have negative values.

In [4]:
df.loc[df['broadband_pe'] <0, 'broadband_pe'] = np.nan,
df.loc[df['gini_ind_inequality_e'] <0, 'gini_ind_inequality_e'] = np.nan,
df.loc[df['no_veh_pe'] <0, 'no_veh_pe'] = np.nan

Generate new variables for display in the maps.

In [5]:
df['GEOID'] = df['geoid_tract_20']
df['Percentage of population below poverty line'] = df['blw_pov_lvl_pe']
df['Depression rate'] = df['depression_rate']
df['Percentage of households without health insurance'] = df['no_health_ins_pe']
df['Gini inequality index'] = df['gini_ind_inequality_e']
df['Percentage of households without vehicles'] = df['no_veh_pe']
df['Percentage of Black residents'] = df['nh_black_alone_pe']
df['Percentage of residents with internet access'] = df['broadband_pe']
df['Percentage of residents with a high school diploma'] = df['hs_grad_pe']
df['Walkability score'] = df['median_natwalkind']

df['GEOID'] = df['GEOID'].astype(str)

Join the data with zip codes.

In [6]:
zcta = pd.read_csv('https://www2.census.gov/geo/docs/maps-data/data/rel2020/zcta520/tab20_zcta520_tract20_natl.txt',
                   sep='|')

zcta.columns = zcta.columns.str.lower()
zcta['geoid_tract_20'] = zcta['geoid_tract_20'].astype(str)
zcta = zcta.dropna(subset=['geoid_zcta5_20'])
    
zcta = zcta[['geoid_tract_20','namelsad_zcta5_20']]
zcta.rename(columns={'geoid_tract_20':'GEOID','namelsad_zcta5_20':'Zip Code'},inplace=True)
    
zcta['Zip Code'] = [zip_code.split(' ')[1] for zip_code in zcta['Zip Code']]

In [7]:
df = df.merge(zcta, how = 'left', on = 'GEOID', indicator=True)
    
df.drop(columns='_merge',inplace=True)

Add data on the Wards associated with each zip code.

In [8]:
zips = pd.read_csv('../StartingWithToday/data/zips.csv')
    
df = df.merge(zips[['Zip Code','Ward']], how='left', on = 'Zip Code', indicator=True)

Format features in the data frame for display.

In [9]:
for var in [['Percentage of population below poverty line', 'Depression rate',
             'Percentage of households without health insurance',
             'Gini inequality index', 'Percentage of households without vehicles',
             'Percentage of Black residents','Percentage of residents with internet access',
             'Percentage of residents with a high school diploma','Walkability score','Zip Code','Ward']]:
    if var != 'Zip Code' and var != 'Ward':
        df[var] = round(df[var],2)
    if var != 'Zip Code' and var != 'Ward' and var != 'Needs Score':
        df[var] = df[var].astype(str)
        
for var in ['Gini inequality index', 'Percentage of households without vehicles',
            'Percentage of residents with internet access']:
    df.loc[df[var] == 'nan', var] = 'Data Not Available'

In [10]:
df.rename(columns={'Ward':'Ward/County'},inplace=True)
    
df.loc[df['countyname'] == "Prince George's", 'Ward/County'] = "Prince George's"
    
df.loc[df['Ward/County'] == "nan", 'Ward/County'] = "Data Not Available"

Export data for display in map.

In [11]:
df[['GEOID','Percentage of population below poverty line', 'Depression rate',
    'Percentage of households without health insurance','Gini inequality index',
    'Percentage of households without vehicles','Percentage of Black residents',
    'Percentage of residents with internet access','Percentage of residents with a high school diploma',
    'Walkability score', 'Zip Code', 'Ward/County','countyname']].drop_duplicates().to_csv('./data/display-data.csv',
                                                                                           index=False)

In [12]:
df = df.drop_duplicates()

Create a function to generate z-scores of the features needed in the needs score.

In [13]:
def update_df(county, df = df):
    df_cnty = df[df['countyname'] == county]
    df_cnty['hs_grad_pe_gap'] = df_cnty['hs_grad_pe'].max() - df_cnty['hs_grad_pe']
    df_cnty['broadband_pe_gap'] = df_cnty['broadband_pe'].max() - df_cnty['broadband_pe']
    df_cnty['median_natwalkind_gap'] = df_cnty['median_natwalkind'].max() - df_cnty['median_natwalkind']
    
    features = ['blw_pov_lvl_pe','depression_rate','no_health_ins_pe','gini_ind_inequality_e',
                'no_veh_pe','hs_grad_pe_gap','broadband_pe_gap','median_natwalkind_gap']
    
    df_cnty[['z_blw_pov_lvl_pe','z_depression_rate',
             'z_no_health_ins_pe','z_gini_ind_inequality_e',
             'z_no_veh_pe','z_hs_grad_pe_gap',
             'z_broadband_pe_gap','z_median_natwalkind_gap']] = StandardScaler().fit_transform(df_cnty[features])
    
    return df_cnty

Export the data to a CSV to be ingested by a Python script.

In [14]:
update_df('District of Columbia').to_csv('../StartingWithToday/data/needs_score_dc.csv',index=False)

update_df("Prince George's").to_csv('../StartingWithToday/data/needs_score_md.csv',index=False)